<a href="https://colab.research.google.com/github/esonwong/learning-python/blob/main/%E8%87%AA%E5%AE%9A%E4%B9%89%E7%9F%A5%E8%AF%86%E5%BA%93%E8%81%8A%E5%A4%A9%E6%9C%BA%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介
你只需要按照这份教程一步一步操作，即可实现一个「自定义知识库的聊天机器人」。 本教程的使用方法很简单，你只需要点击代码前的播放按钮即可。

## 安装依赖库

点击下方代码前的播放按钮即可，这一步主要是安装好后续程序依赖的库。

In [ ]:
!pip3 install llama-index
!pip3 install ipywidgets

## 引入模块

In [ ]:
import ipywidgets as widgets
import os
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from IPython.display import Markdown, display

## 导入个人知识库数据

1. 左侧文件夹菜单中新建 `data` 文件夹。

2. 上传知识库文件到文件夹内。

## 设置 OpenAI API Key

点击代码前的播放按钮，运行代码后，在框内输入你的个人 OpenAI API Key，黏贴完记得按下回车键。

In [ ]:
os.environ["OPENAI_API_KEY"] = input("将你的 OpenAI API Key 黏贴到这里：")

## 构建索引 
下面用 OpenAI 的 embedding API 来构建索引，这个 API 可以将文本转换为向量，这样就可以通过计算向量的相似度来判断两个文本主义上的相似度。然后把这些文本交给 GPT 来生成答案。

In [ ]:
documents = SimpleDirectoryReader('data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)
index.storage_context.persist()

## 加载向量索引 index.json

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir='./storage')
index = load_index_from_storage(storage_context)

## 输入问题

执行下面的代码显示输入框。

In [ ]:
textarea = widgets.Textarea(
    value='### Question\n我的产品折断了。',
    placeholder='Type something',
    description='String:',
    disabled=False,
    layout=widgets.Layout(width='80%', height='200px')
)
display(textarea)

## 查询问题

点击执行按钮查询生成回答。

In [ ]:
query = textarea.value
query_engine = index.as_query_engine()
response = query_engine.query(query)
display(Markdown(response.response))